In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    #'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
    #             'rnd_padding', 
                 'aug_padding',
                "zoom_padding"
]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/1conv/64filts_size5/' #'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 64 #None
kernel_size = [5] #None
pool_size = None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
#architecture = "only_denses"
architecture = "conv_dense"

### Loading stuff

In [3]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [4]:
#model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
model = building_1convdense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 313)         20345     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 76)          0         
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
dense_4 (Dense)              (None, 1000, 313)         20345     
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 76)          0         
__________


Epoch 00026: val_acc improved from 0.15477 to 0.16586, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/1conv/64filts_size5/task2/aug_padding/0/weights-improvement-026-0.1659.hdf5
Epoch 27/200
967/967 [==============================] - 27s 28ms/step - loss: 0.5084 - acc: 0.2857 - val_loss: 0.1899 - val_acc: 0.1716

Epoch 00027: val_acc improved from 0.16586 to 0.17159, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/1conv/64filts_size5/task2/aug_padding/0/weights-improvement-027-0.1716.hdf5
Epoch 28/200
967/967 [==============================] - 27s 28ms/step - loss: 0.5029 - acc: 0.2895 - val_loss: 0.1844 - val_acc: 0.1648

Epoch 00028: val_acc did not improve from 0.17159
Epoch 29/200
967/967 [==============================] - 27s 28ms/step - loss: 0.4929 - acc: 0.2918 - val_loss: 0.1763 - val_acc: 0.1643

Epoch 00029: val_acc did not improve from 0.17159
Epoch 30/200
967/967 [==============================] - 27s 28ms/step - loss

967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00067: val_acc did not improve from 0.18402
Epoch 68/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00068: val_acc did not improve from 0.18402
Epoch 69/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00069: val_acc did not improve from 0.18402
Epoch 70/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00070: val_acc did not improve from 0.18402
Epoch 71/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00071: val_acc did not improve from 0.18402
Epoch 72/200
967/967 [==============================] - 27s 28ms/step - loss: 

967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00111: val_acc did not improve from 0.18402
Epoch 112/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00112: val_acc did not improve from 0.18402
Epoch 113/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00113: val_acc did not improve from 0.18402
Epoch 114/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00114: val_acc did not improve from 0.18402
Epoch 115/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00115: val_acc did not improve from 0.18402
Epoch 116/200
967/967 [==============================] - 27s 28ms/step - l


Epoch 00155: val_acc did not improve from 0.18402
Epoch 156/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00156: val_acc did not improve from 0.18402
Epoch 157/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00157: val_acc did not improve from 0.18402
Epoch 158/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00158: val_acc did not improve from 0.18402
Epoch 159/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00159: val_acc did not improve from 0.18402
Epoch 160/200
967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00160: val_acc did not improve from 0.18402
Epoch 161

967/967 [==============================] - 27s 28ms/step - loss: 6.0132 - acc: 0.0893 - val_loss: 1.8774 - val_acc: 0.0833

Epoch 00200: val_acc did not improve from 0.18402
It has been  1:29:03.275140
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1000, 64)          8384      
_________________________________________________________________
dense_7 (Dense)              (None, 1000, 313)         20345     
_________________________________________________________________
dropout_8 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_

967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00035: val_acc did not improve from 0.09832
Epoch 36/200
967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00036: val_acc did not improve from 0.09832
Epoch 37/200
967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00037: val_acc did not improve from 0.09832
Epoch 38/200
967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00038: val_acc did not improve from 0.09832
Epoch 39/200
967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00039: val_acc did not improve from 0.09832
Epoch 40/200
967/967 [==============================] - 27s 28ms/step - loss: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00075: val_acc did not improve from 0.09832
Epoch 76/200
967/967 [==============================] - 27s 27ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00076: val_acc did not improve from 0.09832
Epoch 77/200
967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00077: val_acc did not improve from 0.09832
Epoch 78/200
967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00078: val_acc did not improve from 0.09832
Epoch 79/200
967/967 [==============================] - 27s 28ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00079: val_acc did not improve from 0.09832
Epoch 80/200
967/967 [==============================] - 27s 28ms/step - loss: 

967/967 [==============================] - 26s 27ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00119: val_acc did not improve from 0.09832
Epoch 120/200
967/967 [==============================] - 26s 27ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00120: val_acc did not improve from 0.09832
Epoch 121/200
967/967 [==============================] - 26s 27ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00121: val_acc did not improve from 0.09832
Epoch 122/200
967/967 [==============================] - 26s 27ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00122: val_acc did not improve from 0.09832
Epoch 123/200
967/967 [==============================] - 26s 27ms/step - loss: 4.6260 - acc: 0.1800 - val_loss: 1.6554 - val_acc: 0.0983

Epoch 00123: val_acc did not improve from 0.09832
Epoch 124/200
967/967 [==============================] - 26s 27ms/step - l